# Day 1 — HP Model (6-Residue Lattice Toy)

**Objective**  
- Enumerate **all 284** valid self-avoiding folds for a 6-residue chain on a 2D lattice  
- Score energies using **H–H contacts** (non-consecutive neighbors only)  
- Visualize best folds and the **energy distribution**

> You can change the sequence in the **Setup** cell below.

## Setup

In [ ]:
# Imports & basic config
import os
import math
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt

# Do not set colors or styles here (keeps defaults as requested)
plt.rcParams.update({"figure.dpi": 120})

# Output directory for figures
FIG_DIR = "figures"
os.makedirs(FIG_DIR, exist_ok=True)

# Default 6-residue sequence (editable)
SEQ = ['H','P','H','H','P','H']

# Lattice moves (R, L, U, D)
MOVES = [(1,0), (-1,0), (0,1), (0,-1)]

print("Sequence:", ''.join(SEQ))

## HP Model Essentials (Quick Notes)
- Grid: 2D square lattice; residues occupy lattice points.
- Valid fold: a **self-avoiding walk (SAW)** — no coordinate is visited more than once.
- Energy rule: count **non-consecutive** `H–H` contacts at **Manhattan distance 1**.  
  Each such contact contributes **-1** to the energy (more negative = better).
- We enumerate **all four** possible first steps to ensure the **full set of 284** folds.

## Enumeration of Self-Avoiding Walks

In [ ]:
from typing import List, Tuple, Set

Coord = Tuple[int, int]

def _dfs(path: List[Coord], visited: Set[Coord], n_steps: int) -> List[List[Coord]]:
    if len(path) == n_steps + 1:
        return [path]
    x, y = path[-1]
    results = []
    for dx, dy in MOVES:
        nx, ny = x + dx, y + dy
        if (nx, ny) not in visited:
            results.extend(_dfs(path + [(nx, ny)], visited | {(nx, ny)}, n_steps))
    return results

def enumerate_saws_all(n_steps: int) -> List[List[Coord]]:
    """Enumerate all SAWs with all four possible first steps."
    paths = []
    for first in MOVES:
        start = [(0,0), (first[0], first[1])]
        paths.extend(_dfs(start, set(start), n_steps))
    return paths

# For 6 residues, we have 5 steps
N_STEPS = 5
folds = enumerate_saws_all(N_STEPS)
print("Total folds (should be 284):", len(folds))

## Energy Function (H–H Contacts)

In [ ]:
def manhattan(a: Coord, b: Coord) -> int:
    return abs(a[0]-b[0]) + abs(a[1]-b[1])

def score_energy(seq, coords) -> int:
    n = len(coords)
    e = 0
    for i in range(n):
        for j in range(i+1, n):
            if abs(i - j) > 1 and seq[i] == 'H' and seq[j] == 'H':
                if manhattan(coords[i], coords[j]) == 1:
                    e -= 1
    return e

energies = [score_energy(SEQ, c) for c in folds]
dist = Counter(energies)
print("Energy distribution:", dict(sorted(dist.items())))
print("Best (min) energy:", min(energies))

## Visualization

In [ ]:
def draw_fold(coords, seq, title=None, save_path=None):
    xs = [c[0] for c in coords]
    ys = [c[1] for c in coords]

    fig = plt.figure()
    plt.plot(xs, ys, marker='o', linestyle='-')  # defaults; do not set colors
    # Different markers for H/P (no colors specified)
    for (x, y), t in zip(coords, seq):
        if t == 'H':
            plt.scatter([x], [y], marker='o')
        else:
            plt.scatter([x], [y], marker='s')

    # Grid and formatting
    plt.grid(True, which="both", linestyle='--', linewidth=0.5)
    plt.gca().set_aspect('equal', 'box')
    if title:
        plt.title(title)
    plt.xlabel("x")
    plt.ylabel("y")
    plt.tight_layout()
    if save_path:
        plt.savefig(save_path, bbox_inches='tight')
    plt.show()

def plot_energy_hist(energies, save_path=None):
    fig = plt.figure()
    plt.hist(energies, bins=range(min(energies), max(energies)+2))
    plt.title("Energy Distribution")
    plt.xlabel("Energy")
    plt.ylabel("Count")
    plt.tight_layout()
    if save_path:
        plt.savefig(save_path, bbox_inches='tight')
    plt.show()

In [ ]:
# Rank by energy (ascending)
ranked = sorted(zip(energies, folds), key=lambda x: x[0])
best_e = ranked[0][0]
print("Best energy:", best_e)

# Plot top-k folds (default k=5, including ties capped at k)
k = 5
top_k = ranked[:k]

paths = []
for idx, (e, coords) in enumerate(top_k, start=1):
    sp = os.path.join(FIG_DIR, f"fold_top_{idx}_E{e}.png")
    draw_fold(coords, SEQ, title=f"Top {idx} (E={e})", save_path=sp)
    paths.append(sp)

# Energy distribution
hist_path = os.path.join(FIG_DIR, "energy_hist.png")
plot_energy_hist(energies, save_path=hist_path)
paths.append(hist_path)

print("Saved figures:")
for p in paths:
    print(" -", p)

### Optional: Contact Map of Best Fold

In [ ]:
def contact_map(seq, coords):
    n = len(coords)
    M = np.zeros((n, n), dtype=int)
    for i in range(n):
        for j in range(n):
            if abs(i - j) > 1 and seq[i]=='H' and seq[j]=='H' and manhattan(coords[i], coords[j]) == 1:
                M[i, j] = 1
    return M

def plot_contact_map(M, save_path=None):
    fig = plt.figure()
    plt.imshow(M, interpolation='nearest')
    plt.title("H–H Contact Map (non-consecutive)")
    plt.xlabel("Residue index")
    plt.ylabel("Residue index")
    plt.tight_layout()
    if save_path:
        plt.savefig(save_path, bbox_inches='tight')
    plt.show()

best_coords = ranked[0][1]
M = contact_map(SEQ, best_coords)
cm_path = os.path.join(FIG_DIR, "contact_map_best.png")
plot_contact_map(M, save_path=cm_path)
print("Saved:", cm_path)